In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sent2vec.vectorizer import Vectorizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
with open("./Dataset/dataframes/df_preprocessed_with_numbers.csv", 'r', encoding="utf8") as f:
  df_preprocessed = pd.read_csv(filepath_or_buffer=f, sep=',')

#with open("/content/drive/MyDrive/Colab Notebooks/apt/Dataset/dataframes/df_preprocessed_without_all_tokens_with_numbers.csv", 'r') as f:
  #df_preprocessed = pd.read_csv(filepath_or_buffer=f, sep=',')

#with open("/content/drive/MyDrive/Colab Notebooks/apt/Dataset/dataframes/df_preprocessed_without_numbers_partially.csv", 'r') as f:
  #df_preprocessed = pd.read_csv(filepath_or_buffer=f, sep=',')

In [7]:
print(df_preprocessed.head(3))

   Unnamed: 0  id                     query  \
0           0   1  bridal shower decoration   
1           1   2      lead christmas light   
2           2   4                 projector   

                                       product_title  \
0        Accent Pillow with Heart Design - Red/Black   
1  Set of 10 Battery Operated Multi LED Train Chr...   
2         ViewSonic Pro8200 DLP Multimedia Projector   

                                 product_description  median_relevance  \
0  Red satin accent pillow embroidered with a hea...                 1   
1  Set of 10 Battery Operated Train Christmas Lig...                 4   
2                                                NaN                 4   

   relevance_variance                                            product  
0               0.000  accent pillow heart design red black red satin...  
1               0.000  set 10 battery operate multi led train christm...  
2               0.471         viewsonic pro8200 dlp multimedia p

In [8]:
queries = df_preprocessed['query']
product = df_preprocessed['product']

In [9]:
df_preprocessed['len_q'] = [len(queries[x].split()) for x in range(0, len(queries))] 
df_preprocessed['len_p'] = [len(product[x].split()) for x in range(0, len(product))]

In [10]:
df_preprocessed = df_preprocessed.loc[(df_preprocessed['len_q'] < 512) & (df_preprocessed['len_p'] < 512)]

In [11]:
queries = df_preprocessed['query']
product = df_preprocessed['product']
classes = df_preprocessed['median_relevance']

In [12]:
print(len(queries))
print(len(product))
print(len(classes))

10139
10139
10139


In [13]:
sents1 = [x for x in queries]
sents2 = [x for x in product]
classses = [x for x in classes]

In [ ]:
vectorizer = Vectorizer()
vectorizer.run(sents1)
vectors = vectorizer.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
vectorizer = Vectorizer()
vectorizer.run(sents2[:3000])
vectors2 = vectorizer.vectors

In [ ]:
vectorizer = Vectorizer()
vectorizer.run(sents2[3000:])
vectors22 = vectorizer.vectors

In [10]:
!pip install podium-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from podium import Vocab, Field, LabelField
from podium.datasets import TabularDataset
from podium.vectorizers import GloVe

In [16]:
train = pd.DataFrame()
train['query'] = sents1
train['product'] = sents2
train['label'] = classes

In [17]:
def lowercase(text):
    return text.lower()

max_vocab_size = 10_000
vocab = Vocab(max_size=max_vocab_size, min_freq=2)

S1 = Field('query', numericalizer=vocab, pretokenize_hooks=[lowercase])
S2 = Field('product', numericalizer=vocab, pretokenize_hooks=[lowercase])
LABEL = LabelField('label')

fields = [
    S1,
    S2,
    LABEL,
]

train = TabularDataset.from_pandas(train, fields)
train.finalize_fields()

glove = GloVe()
# Load only the vectors of vocab words.
embeddings = glove.load_vocab(vocab)

# Generate padded batch.
train_batch = train.batch(add_padding=True)

c:\users\maria\appdata\local\programs\python\python39\lib\site-packages\podium\vocab.py:514: UserWarning: Vocabulary is finalized already. This should be used only if multiple fields use same vocabulary.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████| 862M/862M [2:48:21<00:00, 85.4kB/s]


In [18]:
def cosine_similarity(a, b):
    """
    Receives two 2D numpy arrays and calculates cosine similarity across the second axis.
    For examples, if `a` and `b` have shape (32, 10), the resulting array should have shape (32,).
    
    Returns:
        1D numpy array with cosine similarities
    """
    return [np.dot(a[i,:],b[i,:])/(np.linalg.norm(a[i,:])*np.linalg.norm(b[i,:])) for i in range(len(a))]

In [19]:
print(train_batch.product.shape)

(10139, 503)


In [20]:
query_train = np.array([[embeddings[index] for index in query] for query in train_batch.query])
product_train = np.array([[embeddings[index] for index in product] for product in train_batch.product])

MemoryError: Unable to allocate 11.4 GiB for an array with shape (10139, 503, 300) and data type float64

In [ ]:
query_train_mean = np.array([np.mean(x, axis = 0) for x in query_train])
product_train_mean = np.array([np.mean(x, axis = 0) for x in product_train])

In [ ]:
print(query_train_mean.shape)
print(product_train_mean.shape)

In [ ]:
import json

In [ ]:
query_vector = query_train_mean.tolist()
product_vector = product_train_mean.tolist()
labels = np.array(classes).tolist()

In [ ]:
with open('glove_with_numbers.json', 'w') as outfile:
  json.dump([query_vector, product_vector, labels], outfile,indent=3)
#with open('embedding_without_numbers.json', 'w') as outfile:
  #json.dump([query_vector, product_vector, labels], outfile,indent=3)
#with open('embedding_without_numbers_partially.json', 'w') as outfile:
  #json.dump([query_vector, product_vector, labels], outfile,indent=3)